In [1]:
!uv add youtube-transcript-api

Resolved 172 packages in 6.15s
Installed 1 package in 380ms
 + youtube-transcript-api==1.2.4


In [1]:
video_id = 'r0gHbW_MmaQ'

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi

In [21]:
ytt_api = YouTubeTranscriptApi()
transcript = ytt_api.fetch(video_id)

In [8]:
transcript.snippets[-1]

FetchedTranscriptSnippet(text='you. Thanks. See you.', start=3526.079, duration=4.52)

In [10]:
def format_timestamp(seconds: float) -> str:
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes:02}:{secs:02}"


In [11]:
format_timestamp(3526.079)

'58:46'

In [14]:
transcript_lines = []

for snippet in transcript.snippets:
    ts = format_timestamp(snippet.start)
    line = f'{ts} {snippet.text}'
    transcript_lines.append(line)

In [19]:
subtitles = '\n'.join(transcript_lines[:10])

In [20]:
print(subtitles[:400])

00:00 Hi everyone, welcome to the second
00:02 iteration of AI engineering boot camp.
00:05 So I renamed the course recently. It
00:08 used to be called AI boot camp but there
00:10 are so many boot camps and AI boot camps
00:12 these days. So I was thinking if there's
00:15 a way to somehow be different from these
00:17 courses and uh maybe you know me by uh
00:21 from courses u from free courses


In [22]:
def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)

subtitles = make_subtitles(transcript)

In [32]:
from pathlib import Path

subtitles_file = Path(f'subtitles_{video_id}.txt')
subtitles_file.write_text(subtitles, encoding='utf-8')

56264

In [4]:
from pathlib import Path

subtitles_file = Path(f'subtitles_{video_id}.txt')
subtitles = subtitles_file.read_text(encoding='utf-8')

In [5]:
from openai import OpenAI
openai_client = OpenAI()

In [6]:
def llm(instructions, prompt, model='gpt-4o-mini'):
    messages = [
        {'role': 'system', 'content': instructions},
        {'role': 'user', 'content': prompt}
    ]

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text


In [69]:
instructions = """
Summarize the following youtube transcript and create a timestamped
list of video chaptes

The goal is to use this as the description under the YouTube video.
The output should be copy-pastable to youtube, so don't start the
summary with "the transcript provides...". 
Don't start with "Welcome...", "In this video," etc

It should contain 2-3 paragraphs of text.

Don't use any formatting in the response.

The correct name of the course is "AI Engineering Buildcamp" 

After the description, create the chapter list in the following format

<timestamp> <chapter name>

Make sure chapters are at least 3-4 minutes long but not longer than
6 minutes. The chapters should 
correspond to what's actually discussed in the video

The chapter should start where the concept is introduced, and should be very
precise - this is going to be used for youtube, and viewers will use it for
quick navigation around the video
"""

In [72]:
summary = llm(
    instructions=instructions,
    prompt=subtitles,
    model='gpt-4o-mini'
)

In [73]:
print(summary)

The AI Engineering Buildcamp focuses on a hands-on approach to building AI systems, emphasizing the engineering mindset. Participants are encouraged to engage in practical programming while adhering to best practices. The course structure entails a mix of core and optional materials, ensuring learners can navigate content according to their pace. The instructor introduces the curriculum, highlights the platform used, and addresses how weekly sessions will be organized, including office hours for additional support.

Over the coming weeks, the buildcamp will cover various topics, such as foundational concepts of large language models (LLMs), programming with APIs, implementing structured outputs, and building agents. Participants are to embark on mini projects and ultimately conceptualize their capstone projects, which will showcase their skills. Additionally, there's an emphasis on collaboration through a dedicated Slack community, where learners can share insights and ask questions th

<17:10> <Week 3: Agents and Interactive Learning>

In [8]:
from pydantic import BaseModel, Field

class Chapter(BaseModel):
    timestamp: str
    title: str
    reference_line: str = Field(description="")

class YTSummaryResponse(BaseModel):
    summary: str
    chapters: list[Chapter]

    def print(self):
        print(self.summary)
        print()

        print('Chapters:')

        for c in self.chapters:
            print(f'{c.timestamp} {c.title}')

In [9]:
def llm_structured(instructions, prompt, model='gpt-4o-mini', output=YTSummaryResponse):
    messages = [
        {'role': 'system', 'content': instructions},
        {'role': 'user', 'content': prompt}
    ]

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output
    )

    return response.output_parsed

In [78]:
summary = llm_structured(
    instructions=instructions,
    prompt=subtitles,
    model='gpt-4o-mini'
)

summary.print()

The AI Engineering Buildcamp focuses on practical AI engineering, emphasizing building systems around AI rather than just the AI concepts themselves. The course has been designed to differentiate itself from the many existing AI boot camps. Participants will engage in building applications using an engineering mindset where testability and best practices are paramount. The course structure includes core and optional content, so students can navigate through essential topics while having the freedom to explore more advanced materials based on their interests or project needs.

Throughout the course, students will learn how to interact with large language models (LLMs) like OpenAI as well as incorporate them into various applications. Each week will introduce new concepts and culminate in a capstone project that encourages creative problem-solving. The course also emphasizes collaboration through Slack for peer discussions and brainstorming ideas, laying a solid foundation for developing

In [21]:
instructions = """
You are given a raw YouTube transcript as plain text. Each transcript line is formatted exactly like:

[LINE_INDEX] TIMESTAMP text...

Example:
[0012] 08:03 So this is the library we're going to um

Your job is to return ONLY valid JSON that matches the provided Pydantic schema (YTSummaryResponse).

PRIMARY GOAL
Create a YouTube description + a chapter list that provides FULL COVERAGE of the video:
- Coverage must start at the FIRST timestamp in the transcript.
- Coverage must end at the LAST timestamp in the transcript.
- There must be NO uncovered gaps between chapters.
  (The end of one chapter is the start of the next chapter.)
- Therefore, the first chapter timestamp MUST equal the first transcript timestamp,
  and the last chapter must extend to the end of the video (last transcript timestamp).

OUTPUT FORMAT (STRICT)
- Return ONLY a single JSON object.
- Do not include any additional keys beyond the schema.
- Do not include markdown, code fences, or extra commentary.

SUMMARY RULES
- Write 2–3 paragraphs of plain text (no bullet points, no headings, no markdown).
- Do NOT start with: "Welcome", "In this video", "Today we will", "The transcript...", or similar framing.
- The tone should fit a YouTube description: clear and informative.
- Use the exact course name: "AI Engineering Buildcamp".
- Do not add links unless they are explicitly present in the transcript.

CHAPTER LIST RULES (FULL COVERAGE REQUIRED)
- Chapters must be in chronological order.
- Chapters must provide FULL COVERAGE from the first to the last timestamp as described above.
- Each chapter should start exactly when a new concept/topic is introduced.
- Aim for chapter length of 3–5 minutes.
- Hard constraints: chapters must be at least 3 minutes long and at most 6 minutes long.
- If strict topic boundaries would cause a chapter to be shorter than 3 minutes,
  merge it with a neighboring topic.
- If a topic runs longer than 6 minutes, split it into multiple chapters at sensible subtopic boundaries.
- Create enough chapters to cover the entire duration; do not stop early.

REFERENCE LINE RULES (CRITICAL — NO HALLUCINATIONS)
- For each chapter you MUST choose exactly ONE anchor line from the transcript and copy it verbatim into `reference_line`.
- `reference_line` MUST match one of the provided transcript lines character-for-character,
  including the [LINE_INDEX] prefix, the timestamp, capitalization, filler words, and punctuation.
- Do NOT paraphrase, rewrite, merge multiple lines, or "clean up" the text.
- If you cannot find an exact transcript line to support a chapter start, do NOT include that chapter.
  Instead, choose a different chapter boundary that DOES have an exact supporting line.

TIMESTAMP RULES (CRITICAL)
- `timestamp` MUST be taken from the anchor line you copied into `reference_line`.
- `timestamp` MUST exactly equal the timestamp that appears inside `reference_line`.
- Do NOT invent timestamps. Do NOT approximate. Do NOT shift by a few seconds.
- The first chapter timestamp MUST equal the first transcript timestamp (full coverage).
- Every next chapter timestamp must be later than the previous chapter timestamp.

CHAPTER TITLE RULES
- `title` should be 2–7 words.
- Make it specific and descriptive (avoid vague titles like "More details" or "Next steps").
- No quotes, emojis, or trailing punctuation.

OPTIONAL CHAPTER SUMMARY
- `chapter_summary` may be null or one concise sentence.
- Keep it factual and specific; do not add new information not supported by the transcript.

FINAL SELF-CHECK (DO THIS BEFORE YOU OUTPUT)
1) Identify the first transcript timestamp and ensure the first chapter uses it.
2) Identify the last transcript timestamp and ensure the last chapter reaches it.
3) Ensure there are no gaps: each chapter starts where the previous one ends (i.e., next chapter timestamp is the boundary).
4) Ensure each chapter duration is 3–6 minutes (target 3–5).
5) Ensure every `reference_line` is an exact copy of a transcript line including [LINE_INDEX].
6) Ensure every `timestamp` exactly matches the timestamp inside `reference_line`.

If you cannot satisfy any rule, adjust chapter boundaries and/or increase the number of chapters.

Return JSON only.
"""

In [22]:
from pydantic import BaseModel, Field

class Chapter(BaseModel):
    timestamp: str = Field(
        description="Chapter start timestamp, must match the timestamp in reference_line, format is H:MM:SS or MM:SS."
    )
    title: str = Field(
        description="5-15 word YouTube chapter title."
    )
    reference_line: str = Field(
        description=(
            "A verbatim transcript line INCLUDING its numeric prefix and timestamp, "
            "copied exactly from the provided subtitles. "
            "Must match one of the provided lines character-for-character."
        )
    )


class YTSummaryResponse(BaseModel):
    summary: str
    chapters: list[Chapter]

    def print(self):
        print(self.summary)
        print("\nChapters:\n")
        for c in self.chapters:
            print(f"{c.timestamp} {c.title}")


In [23]:
def preprocess_subtitles(subtitles: str) -> str:
    lines = []
    idx = 0

    for raw in subtitles.split("\n"):
        raw = raw.strip()
        if not raw:
            continue

        lines.append(f"[{idx:04d}] {raw}")
        idx += 1

    return "\n".join(lines)

In [24]:
YTSummaryResponse.model_json_schema()

{'$defs': {'Chapter': {'properties': {'timestamp': {'description': 'Chapter start timestamp, must match the timestamp in reference_line, format is H:MM:SS or MM:SS.',
     'title': 'Timestamp',
     'type': 'string'},
    'title': {'description': '5-15 word YouTube chapter title.',
     'title': 'Title',
     'type': 'string'},
    'reference_line': {'description': 'A verbatim transcript line INCLUDING its numeric prefix and timestamp, copied exactly from the provided subtitles. Must match one of the provided lines character-for-character.',
     'title': 'Reference Line',
     'type': 'string'}},
   'required': ['timestamp', 'title', 'reference_line'],
   'title': 'Chapter',
   'type': 'object'}},
 'properties': {'summary': {'title': 'Summary', 'type': 'string'},
  'chapters': {'items': {'$ref': '#/$defs/Chapter'},
   'title': 'Chapters',
   'type': 'array'}},
 'required': ['summary', 'chapters'],
 'title': 'YTSummaryResponse',
 'type': 'object'}

In [25]:
idx_subtitles = preprocess_subtitles(subtitles)

In [33]:
summary = llm_structured(
    instructions=instructions,
    prompt=idx_subtitles,
    output=YTSummaryResponse,
    # model='gpt-4o-mini',
    model='gpt-5.2',
)

summary.print()

The first live session of AI Engineering Buildcamp explains the course rename and what “buildcamp” means: a hands-on AI engineering approach focused on building testable, production-minded systems around LLMs. The instructor walks through the Maven student portal, how the curriculum is organized by week, what content is core versus optional, and what the first weeks focus on—foundations, RAG, data preparation, search, and structured output.

You’ll get a week-by-week overview of what’s coming next, including agents (with Pentic AI as the main framework and other frameworks as optional), testing and judge-based evaluation, monitoring/observability (OpenTelemetry, Pentic, Clockfire, Grafana), and evaluation with gold-standard datasets. The session also covers how communication works (Slack), how homework and submissions run via the course management platform, why “learning in public” is encouraged, how office hours operate (including time zone caveats), and how to set up your development

In [34]:
summary.chapters[3]

Chapter(timestamp='11:01', title='Agents and frameworks in week 3', reference_line='[0248] 11:01 Um but in addition to that as bonus')

In [35]:
idx_subtitles.split('\n')[238:258]

['[0238] 10:32 implement it we will start talking about',
 '[0239] 10:34 agents so week number three is devoted',
 '[0240] 10:37 to agents so we will what is the',
 '[0241] 10:40 difference between uh traditional rack',
 '[0242] 10:42 and agentic rock and what agents uh what',
 '[0243] 10:46 is an agent. So we are going to cover it',
 "[0244] 10:49 here. Um we're I'm also going to",
 '[0245] 10:52 introduce one main framework that we are',
 '[0246] 10:54 going to use as uh throughout the',
 '[0247] 10:57 course. The framework is called pentki.',
 '[0248] 11:01 Um but in addition to that as bonus',
 '[0249] 11:04 materials as optional materials I will',
 '[0250] 11:06 show you other frameworks. So last',
 '[0251] 11:09 cohort we had two frameworks as like',
 '[0252] 11:12 main frameworks open AAI agents SDK and',
 '[0253] 11:15 Pentic AI and um I found it confusing at',
 '[0254] 11:19 the end. So one of them became optional',
 '[0255] 11:21 but also in addition to open agent SDK',
 "[02